In [5]:
# to make a list of characters
import string

# to do data cleaning and manipulation
import pandas as pd
import numpy as np

# to do text mining, processing, and analysis
import nltk
from nltk import bigrams, trigrams
from nltk.util import ngrams
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.util import bigrams, trigrams, ngrams
from collections import Counter

# to do data visualization
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, classification_report
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch

import googleapiclient.discovery
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

import warnings
warnings.filterwarnings("ignore")

import re

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
import nltk
from nltk.tokenize import word_tokenize
import re

import pandas as pd
from gsdmm import MovieGroupProcess

> Provide your API key, Channel Id or Playlist Id

In [2]:
api_key = "" # fill in your API key here

channel_id = "" # fill in the channel Id here

playlist_id = "" # fill in the playlist Id here

> Scraping Video Data & Comments from a Channel

In [ ]:
# function to get the videos data (id, title, upload date, views, likes, comments count)

def get_channel_videos(api_key, channel_id):
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    try:
        # Get the uploads playlist ID
        channel_response = youtube.channels().list(
            part='contentDetails',
            id=channel_id
        ).execute()
        
        uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']
        
        videos = []
        next_page_token = None
        
        while True:
            playlist_response = youtube.playlistItems().list(
                part='snippet',
                playlistId=uploads_playlist_id,
                maxResults=50,  # 50 is the maximum allowed
                pageToken=next_page_token
            ).execute()
            
            video_ids = [item['snippet']['resourceId']['videoId'] for item in playlist_response['items']]
            
            # Get video details including statistics and snippet for upload date
            video_response = youtube.videos().list(
                part='snippet,statistics',
                id=','.join(video_ids)
            ).execute()
            
            for item in video_response['items']:
                video_id = item['id']
                video_title = item['snippet']['title']
                upload_date = item['snippet']['publishedAt']
                stats = item['statistics']
                videos.append({
                    'video_id': video_id,
                    'title': video_title,
                    'url': f"https://www.youtube.com/watch?v={video_id}",
                    'upload_date': upload_date,
                    'views': int(stats.get('viewCount', 0)),
                    'likes': int(stats.get('likeCount', 0)),
                    'comments': int(stats.get('commentCount', 0)),
                    'favorites': int(stats.get('favoriteCount', 0)),
                    'dislikes': int(stats.get('dislikeCount', 0)) if 'dislikeCount' in stats else None
                })
            
            next_page_token = playlist_response.get('nextPageToken')
            if not next_page_token:
                break
        
        return videos
    
    except HttpError as e:
        print(f'An HTTP error {e.resp.status} occurred:\n{e.content}')
        return []


In [ ]:
# Retrieve video data
video_data = get_channel_videos(api_key, channel_id)

# Convert to DataFrame
df = pd.DataFrame(video_data)

# Save to Excel file
df.to_excel("video data.xlsx", index=False)

# You can also choose different format like .csv 
# df.to_csv("video data.csv", index=False)

# Print a summary
print(f"Total videos retrieved: {len(df)}")
print(df.head())

In [ ]:
# function to get the video comments 

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = api_key

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey=DEVELOPER_KEY)

def getcomments(video):
    try:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video,
            maxResults=100
        )

        comments = []

        # Execute the request.
        response = request.execute()

        # Get the comments from the response.
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            public = item['snippet']['isPublic']
            comments.append([
                comment['authorDisplayName'],
                comment['publishedAt'],
                comment['likeCount'],
                comment['textOriginal'],
                comment['videoId'],
                public
            ])

        while True:
            try:
                nextPageToken = response['nextPageToken']
            except KeyError:
                break
            
            # Create a new request object with the next page token.
            nextRequest = youtube.commentThreads().list(part="snippet", videoId=video, maxResults=100, pageToken=nextPageToken)
            # Execute the next request.
            response = nextRequest.execute()
            # Get the comments from the next response.
            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']
                public = item['snippet']['isPublic']
                comments.append([
                    comment['authorDisplayName'],
                    comment['publishedAt'],
                    comment['likeCount'],
                    comment['textOriginal'],
                    comment['videoId'],
                    public
                ])

        df2 = pd.DataFrame(comments, columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
        return df2
    except HttpError as e:
        if 'commentsDisabled' in str(e):
            print(f"Comments are disabled for video ID: {video}")
            return pd.DataFrame(columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])
        else:
            print(f"An error occurred for video ID {video}: {str(e)}")
            return pd.DataFrame(columns=['author', 'updated_at', 'like_count', 'text','video_id','public'])



In [ ]:
video_ids = df['video_id'].to_list()
print(f"Number of videos: {len(video_ids)}")

In [15]:
# Create an empty list to store all comment DataFrames
all_comments = []

# Iterate through video IDs and collect comments
for video_id in video_ids:
    comments_df = getcomments(video_id)
    if not comments_df.empty:
        all_comments.append(comments_df)

# Combine all comment DataFrames into a single DataFrame
df_all_comments = pd.concat(all_comments, ignore_index=True)

# Now df_all_comments contains all comments from all videos
print(f"Total comments collected: {len(df_all_comments)}")
print(df_all_comments.head())

# Optionally, save to an excel or csv
df_all_comments.to_excel('all_comments.xlsx', index=False)
print("All comments saved to 'all_comments.xlsx'")

df_all_comments.to_excel("all_youtube_comments.xlsx")

# Merge df with df_all_comments based on 'video_id'
df_combined = pd.merge(
    df_all_comments,  # DataFrame with comments
    df[['video_id', 'upload_date', 'title', 'views']],  # DataFrame with video details
    on='video_id',  # Merge on 'video_id'
    how='left'  # Keep all rows from df_all_comments, matching from df
)

# Check the first few rows of the merged DataFrame
print(df_combined.head())

# Optionally, save the combined DataFrame to a .xlsx or .csv
df_combined.to_excel('combined_comments_with_video_details.xlsx', index=False)


> Scraping Video Data and Comments from Playlist

In [ ]:
# Function to get videos from playlist and their details
def get_videos_from_playlist(playlist_id):
    videos = []

    # Fetch the playlist items (videos)
    request = youtube.playlistItems().list(
        part='snippet',
        playlistId=playlist_id,
        maxResults=50  # Max is 50 per request
    )

    while request:
        response = request.execute()

        video_ids = []
        for item in response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            video_ids.append(video_id)

        # Fetch details (title, upload date, and engagement metrics) for each video
        if video_ids:
            video_details_request = youtube.videos().list(
                part='snippet,contentDetails,statistics',
                id=','.join(video_ids)
            )
            video_details_response = video_details_request.execute()

            for video in video_details_response['items']:
                video_data = {
                    'title': video['snippet']['title'],
                    'video_id': video['id'],
                    'upload_date': video['snippet']['publishedAt'],
                    'views': video['statistics'].get('viewCount', 'N/A'),
                    'likes': video['statistics'].get('likeCount', 'N/A'),
                    'comments': video['statistics'].get('commentCount', 'N/A'),
                }
                videos.append(video_data)

        # Check if there's another page of results
        request = youtube.playlistItems().list_next(request, response)

    return videos

# Example usage
video_details = get_videos_from_playlist(playlist_id)

# Convert the result to a DataFrame
df_playlist = pd.DataFrame(video_details)
# Optionally, save the DataFrame to a CSV file
df_playlist.to_excel('playlist_video data.xlsx', index=False)
df_playlist

,title,video_id,upload_date,views,likes,comments
0,Serunya Uji Adrenalin Di Jembatan Rajamandala ...,5gzTcUlCrOI,2024-01-08T05:58:49Z,3688,39,7
1,Keindahan Curug Cikondang Tawarkan Liburan Asi...,XJJ-onCvVAI,2024-01-08T05:58:53Z,7815,89,5
2,Cuci Mata Di Curug Ngebul Gak Bikin Kecewa! | ...,m9DD1ZmcAaQ,2024-01-08T05:59:02Z,2856,46,9
3,Asiknya Main Offroad Di Tengah Kebun Teh Cianj...,tnh8wJWDWyw,2024-01-08T05:58:57Z,2407,33,2
4,Melihat Pemandangan Di Atas Dari Bukit Kasur |...,MR1fYtxO3yY,2024-01-29T08:47:56Z,1276,16,1
...,...,...,...,...,...,...
203,Sekarang Berenang Di Curug Putri Carita Nih! |...,nUzy2eSEkG4,2024-05-06T10:09:00Z,1188,16,3
204,Menikmati Bukit Loloan Yang Ada Di Lombok Utar...,on8MoYfWbHM,2024-05-28T09:39:43Z,396,13,1
205,Merasakan Kesegaran Air Terjun Jagir Di Banyuw...,p6CMX2n7y8M,2024-04-22T07:44:56Z,919,12,2
206,Explore Banten! Kegiatan Alam Menanam Karang P...,sdoFYNBgCFY,2024-05-03T11:53:50Z,414,4,0


In [ ]:
video_ids_playlist = df_playlist['video_id'].to_list()

# Create an empty list to store all comment DataFrames
all_comments = []

# Iterate through video IDs and collect comments
for video_id in video_ids_playlist:
    comments_df = getcomments(video_id)
    if not comments_df.empty:
        all_comments.append(comments_df)

# Combine all comment DataFrames into a single DataFrame
df_all_comments_playlist = pd.concat(all_comments, ignore_index=True)

# Now df_all_comments contains all comments from all videos
print(f"Total comments collected: {len(df_all_comments_playlist)}")
print(df_all_comments_playlist.head())

# Optionally, save to CSV
df_all_comments_playlist.to_excel('all_comments_playlist.xlsx', index=False)
print("All comments saved to 'all_comments_playlist.xlsx'")

# Merge df_jejakpetualang with df_all_comments_playlist based on 'video_id'
df_combined_playlist = pd.merge(
    df_all_comments_playlist,  # DataFrame with comments
    df_playlist[['video_id', 'upload_date', 'title', 'views']],  # DataFrame with video details
    on='video_id',  # Merge on 'video_id'
    how='left'  # Keep all rows from df_all_comments_playlist, matching from df_jejakpetualang
)

# Check the first few rows of the merged DataFrame
print(df_combined_playlist.head())

# Optionally, save the combined DataFrame to a CSV
df_combined_playlist.to_excel('playlist_comments_with_video_details.xlsx', index=False)

> Data Cleaning for Topic Modelling and Sentiment Analysis

In [ ]:
slang_words = pd.read_table("combined_slang_words.txt")
root_words = pd.read_table("combined_root_words.txt")
stop_words = pd.read_table("combined_stop_words.txt")
slang = pd.read_csv("colloquial-indonesian-lexicon.csv")[['slang','formal','In-dictionary']]
slang_dict = slang[['slang','formal']][slang['In-dictionary']==1]
stop_words_list = [item for sublist in stop_words.values.tolist() for item in sublist]

In [19]:
# Initialize Sastrawi (for Bahasa Indonesia) stemmer and stopword remover
stop_factory = StopWordRemoverFactory()
more_stopword = ['dengan', 'ia', 'bahwa', 'oleh', 'aja', 'kalau', 'kalo', 'yg', 'jadi','nya','sy',
                 'yg','ustad','ustadz','ada','sama','semua','haji', 'utk', 'DAN', 'km', 'kmu', 'ak', 
                 'aq', 'akuh', 'dri', 'ada', 'ad', 'di'] # you could add more if you'd like to
stopwords = stop_factory.get_stop_words() + more_stopword + stop_words_list
stopwords = [i for i in stopwords if i != 'tidak']
stopword_remover = stop_factory.create_stop_word_remover()

stem_factory = StemmerFactory()
stemmer = stem_factory.create_stemmer()

# Preprocessing functions
def to_lower(text):
    return text.lower()

def remove_number(text):
    return ''.join(char for char in text if not char.isdigit())

def remove_punctuation(text):
    punctuation = string.punctuation
    return ''.join(char for char in text if char not in punctuation)

def remove_whitespace(text):
    return ' '.join(text.split())

def remove_stopwords(text):
    return ' '.join(word for word in nltk.word_tokenize(text) if word not in stopwords)

def get_bigrams(text):
    tokens = nltk.word_tokenize(text)
    bigram_phrases = [' '.join(bigram) for bigram in bigrams(tokens)]
    return bigram_phrases

def get_trigrams(text):
    tokens = nltk.word_tokenize(text)
    trigram_phrases = [' '.join(trigram) for trigram in trigrams(tokens)]
    return trigram_phrases

def get_fourgrams(text):
    tokens = nltk.word_tokenize(text)
    fourgram_phrases = [' '.join(fourgram) for fourgram in ngrams(tokens, 4)]
    return fourgram_phrases

def safe_preprocess(text):
    if pd.isna(text) or not isinstance(text, str):
        return ""
    return text

def replace_slang(text):
    for row, slang, formal in slang_dict.itertuples():
        text = text.replace(slang, formal)
    return text

def text_preprocessing(df, column):
    # First, safely convert all values to strings and handle NaN
    df[f'{column}_processed'] = df[column].apply(safe_preprocess)
    
    df[f'text_remove_number'] = df[f'{column}_processed'].apply(remove_number)
    df[f'text_lower'] = df[f'text_remove_number'].apply(to_lower)
    df[f'text_remove_punctuation'] = df[f'text_lower'].apply(remove_punctuation)
    df[f'text_remove_stopwords'] = df[f'text_remove_punctuation'].apply(remove_stopwords)
    df[f'text_stemmed'] = df[f'text_remove_stopwords'].apply(stemmer.stem)
    
    return df


['adapun', 'agaknya', 'akan', 'akhir', 'akhirnya', 'akulah', 'amatlah', 'andalah', 'antara', 'apa', 'apabila', 'apalagi', 'artinya', 'asalkan', 'atau', 'ataupun', 'awalnya', 'bagaikan', 'bagaimanakah', 'bagi', 'bahkan', 'bahwasanya', 'bakal', 'balik', 'bapak', 'bawah', 'begini', 'beginikah', 'begitu', 'begitulah', 'bekerja', 'belakangan', 'belumlah', 'benarkah', 'berada', 'berakhirlah', 'berapa', 'berapalah', 'berarti', 'berbagai', 'beri', 'berikut', 'berjumlah', 'berkata', 'berkeinginan', 'berlainan', 'berlangsung', 'bermacam', 'bermaksud', 'bersama', 'bersiap', 'bertanya', 'berturut', 'bertutur', 'berupa', 'betul', 'biasa', 'bila', 'bisa', 'boleh', 'bolehlah', 'bukan', 'bukanlah', 'bulan', 'cara', 'cukup', 'cukuplah', 'dahulu', 'dan', 'dari', 'datang', 'demi', 'demikianlah', 'depan', 'dia', 'diakhirinya', 'diantara', 'diberi', 'diberikannya', 'dibuatnya', 'didatangkan', 'diibaratkan', 'diingat', 'diinginkan', 'dijelaskan', 'dikarenakan', 'dikatakannya', 'diketahui', 'dikira', 'dilalu

In [ ]:
# Usage
# Assuming 'df' is your DataFrame and 'text' is the column with comments

# IMPORTANT
# only choose one option and delete the other based on your source (channelId or playlistId)

df_cleaned = text_preprocessing(df, "text") # this works if you use channel ID

# or use this if you use playlist

df_cleaned = text_preprocessing(df_combined_playlist, "text") # this works if you use playlist ID

> Topic Modelling and Sentiment Analysis

In [22]:
# Download necessary NLTK data
nltk.download('punkt')

# Load your DataFrame
# df = pd.read_csv('your_data.csv')  # Uncomment and modify this line to load your data

# Assuming df has a 'text_stemmed' column with preprocessed text

# Create a simple sentiment lexicon for Bahasa Indonesia
# This is a very basic lexicon and should be expanded for better results

positive_texts = pd.read_table("positive.txt", names=["words"])
positive_words = [item for sublist in positive_texts.values.tolist() for item in sublist]

negative_texts = pd.read_table("negative.txt", names =["words"])
negative_words = [item for sublist in negative_texts.values.tolist() for item in sublist]

# Step 2: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(df_cleaned['text_stemmed'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Rahmani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
# function to calculate sentiment scores
def get_sentiment_score(text, tfidf_vector):
    words = word_tokenize(text.lower())
    score = 0
    for word in words:
        if word in positive_words:
            score += tfidf_vector[tfidf_vectorizer.vocabulary_[word]] if word in tfidf_vectorizer.vocabulary_ else 1
        elif word in negative_words:
            score -= tfidf_vector[tfidf_vectorizer.vocabulary_[word]] if word in tfidf_vectorizer.vocabulary_ else 1
    return score

In [32]:
df_cleaned = df_cleaned.reset_index(drop=True)
df_cleaned['sentiment_score'] = df_cleaned.apply(
    lambda row: get_sentiment_score(row['text_stemmed'], tfidf_matrix[row.name].toarray()[0]), axis=1
)

In [33]:
# Normalize sentiment scores
df_cleaned['sentiment_score_normalized'] = 2 * (df_cleaned['sentiment_score'] - df_cleaned['sentiment_score'].min()) / (df_cleaned['sentiment_score'].max() - df_cleaned['sentiment_score'].min()) - 1

# Classify sentiments based on scores
def classify_sentiment(score):
    if score < -0.1:
        return 'negative'
    elif score > 0.1:
        return 'positive'
    else:
        return 'neutral'

df_cleaned['sentiment_classification'] = df_cleaned['sentiment_score'].apply(classify_sentiment)

df_cleaned.to_excel("sentiment ready comments.xlsx")

In [24]:
# Load your data (assuming 'text' column contains preprocessed text)
df = pd.read_excel("sentiment ready comments.xlsx")

# Convert preprocessed text to list of tokens
def text_to_tokens(text):
    if isinstance(text, str):
        return [word for word in text.split() if len(word) > 2]
    return []

# Apply tokenization to preprocessed comments
df['tokens'] = df['text_stemmed'].apply(text_to_tokens)

# Remove empty lists
df = df[df['tokens'].map(len) > 0]

# Prepare data for GSDMM
# GSDMM requires a list of lists, where each inner list is the tokenized document
documents = df['tokens'].tolist()

# Initialize GSDMM
num_topics = 15 # Number of clusters/topics
gsdmm_model = MovieGroupProcess(K=num_topics, alpha=0.1, beta=0.3, n_iters=30)

# Fit the GSDMM model on the tokenized documents
vocab_length = len(set([word for doc in documents for word in doc]))
gsdmm_model.fit(documents, vocab_length)

# Assign topics to comments
df['dominant_topic'] = df['tokens'].apply(lambda x: gsdmm_model.choose_best_label(x)[0])
df['topic_probability'] = df['tokens'].apply(lambda x: gsdmm_model.choose_best_label(x)[1])

# Print the topics
print("Top 5 words for each topic:")
for topic_id, word_distribution in enumerate(gsdmm_model.cluster_word_distribution):
    if len(word_distribution) == 0:  # Skip empty topics
        continue
    top_words = sorted(word_distribution.items(), key=lambda x: x[1], reverse=True)[:10]
    top_words = [word for word, freq in top_words]
    print(f"Topic {topic_id}: {', '.join(top_words)}")

# Print sample results
print("Sample comments with their dominant topics:")
for _, row in df.sample(min(5, len(df))).iterrows():
    print(f"Preprocessed text: {row['text'][:50]}...")
    print(f"Tokens: {' '.join(row['tokens'])[:50]}...")
    print(f"Dominant Topic: {row['dominant_topic']}")
    print(f"Topic Probability: {row['topic_probability']}")
    print()

# Optionally, group by dominant topic and show average probability
topic_summary = df.groupby('dominant_topic').agg({
    'topic_probability': ['mean', 'count']
}).reset_index()
topic_summary.columns = ['dominant_topic', 'avg_probability', 'comment_count']
print("\nSummary of topics:")
print(topic_summary)

# Optionally, save results
df.to_csv('comments_with_topics_gsdmm.csv', index=False)
print("Results saved to 'comments_with_topics_gsdmm.csv'")


In stage 0: transferred 755 clusters with 15 clusters populated
In stage 1: transferred 622 clusters with 15 clusters populated
In stage 2: transferred 548 clusters with 15 clusters populated
In stage 3: transferred 532 clusters with 14 clusters populated
In stage 4: transferred 481 clusters with 14 clusters populated
In stage 5: transferred 471 clusters with 14 clusters populated
In stage 6: transferred 472 clusters with 14 clusters populated
In stage 7: transferred 456 clusters with 14 clusters populated
In stage 8: transferred 440 clusters with 14 clusters populated
In stage 9: transferred 432 clusters with 14 clusters populated
In stage 10: transferred 452 clusters with 14 clusters populated
In stage 11: transferred 469 clusters with 14 clusters populated
In stage 12: transferred 428 clusters with 14 clusters populated
In stage 13: transferred 391 clusters with 14 clusters populated
In stage 14: transferred 383 clusters with 14 clusters populated
In stage 15: transferred 399 cluste

note:

- Click on scrollable element to see the description of the topics